### Quiz 3. tips 데이터에서 팁을 많이 받으려면 어떤 데이터가 중요한지 인사이트를 찾아내세요
- 정답은 없습니다. 자유롭게 분석해보세요. 
- size는 식사 인원인것 같네요.
- 아래의 절차대로 Pandas를 이용하여 데이터를 분석해 보세요. (예시는 예시일 뿐입니다.)
    - 가설설정 : 예) 주중보다 주말에 팁을 많이 줄것이다
    - 분석과정설정 : 예) 주중 데이터와 주말데이터를 나눠 평균적 팁 금액의 데이터 프레임을 만든다.
    - 결과확인 : 예) 주말과 주중데이터의 평균 팁에는 큰 차이가 없으므로 가설이 틀렸다. 그러므로 다시 가설설정!
- 위의 가설을 여러개 세우셔서 어떤 데이터가 가장 팁에 대한 데이터에 영향을 많이 주는지 찾아보세요. (최소한 3개이상의 가설을 세워서 분석해보세요.)
- 지금까지 배운내용안에서 분석해보세요.
- 가설, 분석과정, 결과 확인에 대한 설명 주석을 달아주세요.
- 적당한 그래프를 그려주시면 더 좋습니다.
- 반별로 제가 판단해서 가장 잘하신 과제는 다른 분들께 공유하며 제가 스타벅스 커피 삽니다.^^

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
tips = sns.load_dataset("tips")  
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#### 가설1. total_bill이 많을수록 팁이 클 것이다. 
#### 가설설정이유: 비싼 음식을 먹을 수록 경제적 여유가 있을 것이라 생각

In [4]:
tips_df = tips.loc[:,["total_bill", "tip"]] # 음식 금액과 팁 금액을 가져옴
tips_df = tips_df.reset_index(drop = True)
tips_df.tail()

,total_bill,tip
239,29.03,5.92
240,27.18,2.00
241,22.67,2.00
242,17.82,1.75
243,18.78,3.00


In [5]:
def ranges (bill):
    return (bill // 10) * 10

In [6]:
tips_df["Ranges"] = tips_df["total_bill"].apply(ranges).astype('int')
tips_df["total_bill"] = tips_df["total_bill"].astype('int')
tips_df.tail() # 음식 가격을 구간별로 설정

,total_bill,tip,Ranges
239,29,5.92,20
240,27,2.00,20
241,22,2.00,20
242,17,1.75,10
243,18,3.00,10


In [7]:
tips_df["Count"] = 1
tips_df2 = tips_df.pivot_table(values=["Count","tip"], index=["Ranges"], aggfunc=np.sum)
tips_df2 # 각 구간별 팁 총액을 구함

,Count,tip
Ranges,,
0,17,31.24
10,130,319.18
20,65,234.50
30,22,92.20
40,9,44.46
50,1,10.00


In [8]:
tips_df2["average"] = round((tips_df2["tip"] / tips_df2["Count"]), 2)
tips_df2 # 팁 총액을 구간별 인원 수로 나눠서 구간별 평균 팁 금액을 도출함

,Count,tip,average
Ranges,,,
0,17,31.24,1.84
10,130,319.18,2.46
20,65,234.50,3.61
30,22,92.20,4.19
40,9,44.46,4.94
50,1,10.00,10.00


수치적으로 봤을 땐, 비싼 음식을 주문한 고객이 평균적으로 더 많은 팁을 주는것을 알 수 있다. 하지만 유의미한 결과인지는 더 분석해야 한다.

#### 가설2. 점심보단 저녁에 팁을 많이 줄 것이다.
#### 가설설정이유: 점심은 상대적으로 저렴하게 먹으려고 하고 저녁은 데이트나 가족행사 등이 많기에 돈을 더 많이 쓸것이라 생각된다. 

In [9]:
tips_df = tips.loc[:,["time", "tip"]] # 시간과 팁 금액을 가져옴
tips_df = tips_df.reset_index(drop = True)
tips_df.tail()

,time,tip
239,Dinner,5.92
240,Dinner,2.00
241,Dinner,2.00
242,Dinner,1.75
243,Dinner,3.00


In [10]:
tips_df["Count"] = 1
tips_df2 = tips_df.pivot_table(values=["Count","tip"], index=["time"], aggfunc=np.sum)
tips_df2 # 점심, 저녁별 팁 금액의 총합을 구함

,Count,tip
time,,
Lunch,68,185.51
Dinner,176,546.07


In [11]:
tips_df2["average"] = round((tips_df2["tip"] / tips_df2["Count"]), 2)
tips_df2 # 점심, 저녁별로 평균 팁을 구함

,Count,tip,average
time,,,
Lunch,68,185.51,2.73
Dinner,176,546.07,3.10


수치적으로 봤을 땐, 저녁이 점심보다 평균적으로 팁을 더 많이 주는 것을 알 수 있다. 하지만 이게 유의미한 차이인지는 모평균 차이검정을 실시해야 한다.

#### 가설3. 여성보단 남성이 팁을 많이 줄 것이다.
#### 가설설정이유: 남성이 주도적인 식사자리가 많을것이고 연인 혹은 가족앞에서 팁을 더 많이 낼 것이라 생각한다.

In [12]:
tips_df = tips.loc[:,["sex", "tip"]] # 성별과 팁 금액을 가져옴
tips_df = tips_df.reset_index(drop = True)
tips_df.tail()

,sex,tip
239,Male,5.92
240,Female,2.00
241,Male,2.00
242,Male,1.75
243,Female,3.00


In [13]:
tips_df["Count"] = 1
tips_df2 = tips_df.pivot_table(values=["Count","tip"], index=["sex"], aggfunc=np.sum)
tips_df2 # 남성, 여성별로 팁 금액의 총합을 구함

,Count,tip
sex,,
Male,157,485.07
Female,87,246.51


In [14]:
tips_df2["average"] = round((tips_df2["tip"] / tips_df2["Count"]), 2)
tips_df2 # 남성, 여성별로 평균 팁을 구함

,Count,tip,average
sex,,,
Male,157,485.07,3.09
Female,87,246.51,2.83


수치적으로 봤을 땐, 남성이 여성보다 평균적으로 팁을 더 많이 주는 것을 알 수 있다. 하지만 이게 유의미한 차이인지는 모평균 차이검정을 실시해야 한다.